In [ ]:
import json 
import pandas as pd
import numpy as np 
import geopy.distance

#package for flattening json in pandas df
from pandas.io.json import json_normalize

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
calc_last_5_games = True
calc_win_loss = True
calc_win_loss_write_all = True
use_team_keys = False
use_team_hot_encoding = False
use_team_home_away_hot_encoding = False



In [ ]:
#load games - json object
with open('games.json') as gamesJsonRaw:
    d = json.load(gamesJsonRaw)

games = json_normalize(d['games'])

In [ ]:
with open('stadiums.json') as stadiumsJsonRaw:
    stadiumsJson = json.load(stadiumsJsonRaw)

stadiums = json_normalize(stadiumsJson['stadiums'])

In [ ]:
stadiums.head(5)

In [ ]:
games.head()

In [ ]:


#games['season'] = games['season'].astype('float')
games['scoreAway'] = games['scoreAway'].astype('uint8')
games['scoreHome'] = games['scoreHome'].astype('uint8')
games['teamAwayId'] = games['teamAwayId'].astype('uint8')
games['teamHomeId'] = games['teamHomeId'].astype('uint8')

#scoreQuarters          object
#teamAwayCode           object
#teamHomeCode           object



In [9]:


# Create a dataframe for the results - same size as dataset
data = pd.DataFrame(index=range(0,len(games)), columns=['gamesPlayedHome', 'gamesPlayedAway'])

# Iterate through every team
for team in games.teamAwayCode.unique():
    gameCount = 0
    homeGameCount = 0;
    
    # Iterate through each game the team is present in.
    for index, game in games[(games['teamAwayCode'] == team) | (games['teamHomeCode'] == team)].iterrows():
        gameCount += 1
        
        # Update game count for team - whether away or home.
        if game.teamAwayCode == team:
            data.loc[index]['gamesPlayedAway'] = gameCount
        else:
            data.loc[index]['gamesPlayedHome'] = gameCount

data['gamesPlayedAway'] = data['gamesPlayedAway'].astype('uint8')
data['gamesPlayedHome'] = data['gamesPlayedHome'].astype('uint8')
            
# Append the results to the dataset
games = pd.merge(games, data, left_index=True, right_index=True)



In [10]:
games[['teamHomeCode', 'teamAwayCode', 'gamesPlayedHome', 'gamesPlayedAway']].sample(5)

,teamHomeCode,teamAwayCode,gamesPlayedHome,gamesPlayedAway
656,CHI,SAC,45,43
810,DAL,ORL,54,56
26,CHA,BOS,3,3
574,GSW,MIA,39,40
755,NYK,CLE,52,49


In [11]:


def calc_percentage(x, y):
    return 0 if y == 0 else x / y



In [12]:
if calc_win_loss == True:
    
    # Create a dataframe for the results - same size as dataset
    data = pd.DataFrame(
        index=range(0,len(games)), 
        columns=[
            'totalGamesHome', 'totalWinsHome', 'homeGamesHome', 'homeWinsHome', 'awayGamesHome', 'awayWinsHome',
            'percentageTotalWinHome', 'percentageHomeWinHome', 'percentageAwayWinHome',
            'totalGamesAway', 'totalWinsAway', 'homeGamesAway', 'homeWinsAway', 'awayGamesAway', 'awayWinsAway',
            'percentageTotalWinAway', 'percentageHomeWinAway', 'percentageAwayWinAway'
        ]
    )

    # Iterate through each game.
    for index, game in games.iterrows():

        # Home Team
        # ---------
        
        historicGames = games[(games['gamesPlayedHome'] < game.gamesPlayedHome) & ((games['teamHomeCode'] == game.teamHomeCode) | (games['teamAwayCode'] == game.teamHomeCode))].sort_values(by='gamesPlayedHome', ascending=True)
       
        homeGamesHome = len(historicGames[(historicGames['teamHomeCode'] == game['teamHomeCode'])])        
        homeWinsHome = len(historicGames[(historicGames['teamHomeCode'] == game['teamHomeCode']) & (games["scoreAway"] < games["scoreHome"])])
        
        awayGamesHome = len(historicGames[(historicGames['teamAwayCode'] == game['teamHomeCode'])])        
        awayWinsHome = len(historicGames[(historicGames['teamAwayCode'] == game['teamHomeCode']) & (games["scoreAway"] > games["scoreHome"])])

        #totalGames = len(historicGames);
        #totalWins = len(
        #    historicGames[
        #        (historicGames['teamHomeCode'] == game['teamHomeCode']) & (games["scoreAway"] < games["scoreHome"]) |
        #        (historicGames['teamAwayCode'] == game['teamHomeCode']) & (games["scoreAway"] > games["scoreHome"])
        #    ]
        #)
        
        totalGamesHome = homeGamesHome + awayGamesHome
        totalWinsHome = homeWinsHome + awayWinsHome
        
        percentageTotalWinHome = round(calc_percentage(totalWinsHome, totalGamesHome), 3)
        percentageHomeWinHome = round(calc_percentage(homeWinsHome, homeGamesHome), 3)
        percentageAwayWinHome = round(calc_percentage(awayWinsHome, awayGamesHome), 3)
        
        # print('HOME', game['gamesPlayedHome'], 'TG', totalGamesHome, 'W', totalWinsHome, 'HGP', homeGamesHome, 'HW', homeWinsHome, 'AGP', awayGamesHome, 'AW', awayWinsHome)
        # print('HOME', 'W%', percentageTotalWinHome, 'HW%', percentageHomeWinHome, 'AW%', percentageAwayWinHome)
        
        # Away Team
        # ---------
        
        historicGames = games[(games['gamesPlayedAway'] < game.gamesPlayedAway) & ((games['teamAwayCode'] == game.teamAwayCode) | (games['teamHomeCode'] == game.teamAwayCode))].sort_values(by='gamesPlayedAway', ascending=True)
       
        homeGamesAway = len(historicGames[(historicGames['teamHomeCode'] == game['teamAwayCode'])])        
        homeWinsAway = len(historicGames[(historicGames['teamHomeCode'] == game['teamAwayCode']) & (games["scoreAway"] < games["scoreHome"])])
        
        awayGamesAway = len(historicGames[(historicGames['teamAwayCode'] == game['teamAwayCode'])])        
        awayWinsAway = len(historicGames[(historicGames['teamAwayCode'] == game['teamAwayCode']) & (games["scoreAway"] > games["scoreHome"])])

        #totalGames = len(historicGames);
        #totalWins = len(
        #    historicGames[
        #        (historicGames['teamHomeCode'] == game['teamHomeCode']) & (games["scoreAway"] < games["scoreHome"]) |
        #        (historicGames['teamAwayCode'] == game['teamHomeCode']) & (games["scoreAway"] > games["scoreHome"])
        #    ]
        #)
        
        totalGamesAway = homeGamesAway + awayGamesAway
        totalWinsAway = homeWinsAway + awayWinsAway
        
        percentageTotalWinAway = round(calc_percentage(totalWinsAway, totalGamesAway), 3)
        percentageHomeWinAway = round(calc_percentage(homeWinsAway, homeGamesAway), 3)
        percentageAwayWinAway = round(calc_percentage(awayWinsAway, awayGamesAway), 3)
        
        # print('AWAY', game['gamesPlayedAway'], 'TG', totalGamesAway, 'W', totalWinsAway, 'HGP', homeGamesAway, 'HW', homeWinsAway, 'AGP', awayGamesAway, 'AW', awayWinsAway)
        # print('AWAY', 'W%', percentageTotalWinAway, 'HW%', percentageHomeWinAway, 'AW%', percentageAwayWinAway)
        
        # ---------    
            
        # Update the row with the history
        data.loc[index] = [
            totalGamesHome, totalWinsHome, homeGamesHome, homeWinsHome, awayGamesHome, awayWinsHome,
            percentageTotalWinHome, percentageHomeWinHome, percentageAwayWinHome,
            totalGamesAway, totalWinsAway, homeGamesAway, homeWinsAway, awayGamesAway, awayWinsAway,
            percentageTotalWinAway, percentageHomeWinAway, percentageAwayWinAway
        ]

    # Add results to the dataset
    if (calc_win_loss_write_all == True):
        games = pd.merge(games, data, left_index=True, right_index=True)
        games[['teamHomeCode', 'teamAwayCode', 'percentageTotalWinHome', 'percentageHomeWinHome', 'percentageAwayWinHome', 'percentageTotalWinAway', 'percentageHomeWinAway', 'percentageAwayWinAway']].sample(5)
    else:
        games = pd.merge(games, data[['percentageHomeWinHome', 'percentageAwayWinAway']], left_index=True, right_index=True)
        games[['teamHomeCode', 'teamAwayCode', 'percentageHomeWinHome', 'percentageAwayWinAway']].sample(5)

/home/gingasu/.local/lib/python2.7/site-packages/ipykernel_launcher.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/gingasu/.local/lib/python2.7/site-packages/ipykernel_launcher.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/gingasu/.local/lib/python2.7/site-packages/ipykernel_launcher.py:52: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/gingasu/.local/lib/python2.7/site-packages/ipykernel_launcher.py:55: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [13]:
if calc_last_5_games == True:
    
    # Create a dataframe for the results - same size as dataset
    data = pd.DataFrame(index=range(0,len(games)), columns=['lastGame1WinHome', 'lastGame1AtHomeHome', 'lastGame2WinHome', 'lastGame2AtHomeHome', 'lastGame3WinHome', 'lastGame3AtHomeHome', 'lastGame4WinHome', 'lastGame4AtHomeHome', 'lastGame5WinHome', 'lastGame5AtHomeHome'])

    # Iterate through each game where the teams have played at least 5 games.
    for index, game in games.iterrows():

        # Get the last five games for the team.
        last5games = games[(games['gamesPlayedHome'] < game.gamesPlayedHome) & ((games['teamAwayCode'] == game.teamHomeCode) | (games['teamHomeCode'] == game.teamHomeCode))].sort_values(by='gamesPlayedHome', ascending=False).head(5)

        if len(last5games) == 5:
            lastGame1WinHome = last5games["scoreAway"].iloc[0] < last5games["scoreHome"].iloc[0]
            lastGame2WinHome = last5games["scoreAway"].iloc[1] < last5games["scoreHome"].iloc[1]
            lastGame3WinHome = last5games["scoreAway"].iloc[2] < last5games["scoreHome"].iloc[2]
            lastGame4WinHome = last5games["scoreAway"].iloc[3] < last5games["scoreHome"].iloc[3]
            lastGame5WinHome = last5games["scoreAway"].iloc[4] < last5games["scoreHome"].iloc[4]
            #print(lastGame1WinHome, lastGame2WinHome, lastGame3WinHome, lastGame4WinHome, lastGame5WinHome)

            lastGame1AtHomeHome = last5games["teamHomeCode"].iloc[0] == game.teamHomeCode
            lastGame2AtHomeHome = last5games["teamHomeCode"].iloc[1] == game.teamHomeCode
            lastGame3AtHomeHome = last5games["teamHomeCode"].iloc[2] == game.teamHomeCode
            lastGame4AtHomeHome = last5games["teamHomeCode"].iloc[3] == game.teamHomeCode
            lastGame5AtHomeHome = last5games["teamHomeCode"].iloc[4] == game.teamHomeCode
            #print(lastGame1AtHomeHome, lastGame2AtHomeHome, lastGame3AtHomeHome, lastGame4AtHomeHome, lastGame5AtHomeHome)

            # Update the row with the history
            data.loc[index] = [lastGame1WinHome, lastGame1AtHomeHome, lastGame2WinHome, lastGame2AtHomeHome, lastGame3WinHome, lastGame3AtHomeHome, lastGame4WinHome, lastGame4AtHomeHome, lastGame5WinHome, lastGame5AtHomeHome]

    # Convert types 
    data['lastGame1WinHome'] = data['lastGame1WinHome'].astype('bool')
    data['lastGame2WinHome'] = data['lastGame2WinHome'].astype('bool')
    data['lastGame3WinHome'] = data['lastGame3WinHome'].astype('bool')
    data['lastGame4WinHome'] = data['lastGame4WinHome'].astype('bool')
    data['lastGame5WinHome'] = data['lastGame5WinHome'].astype('bool')
    data['lastGame1AtHomeHome'] = data['lastGame1AtHomeHome'].astype('bool')
    data['lastGame2AtHomeHome'] = data['lastGame2AtHomeHome'].astype('bool')
    data['lastGame3AtHomeHome'] = data['lastGame3AtHomeHome'].astype('bool')
    data['lastGame4AtHomeHome'] = data['lastGame4AtHomeHome'].astype('bool')
    data['lastGame5AtHomeHome'] = data['lastGame5AtHomeHome'].astype('bool')

    # Add results to the dataset
    games = pd.merge(games, data, left_index=True, right_index=True)
    
    games[['teamHomeCode', 'teamAwayCode', 'lastGame1WinHome', 'lastGame1AtHomeHome', 'lastGame2WinHome', 'lastGame2AtHomeHome', 'lastGame3WinHome', 'lastGame3AtHomeHome', 'lastGame4WinHome', 'lastGame4AtHomeHome', 'lastGame5WinHome', 'lastGame5AtHomeHome']].sample(5)

In [14]:
if calc_last_5_games == True:
    
    # Create a dataframe for the results - same size as dataset
    data = pd.DataFrame(index=range(0,len(games)), columns=['lastGame1WinAway', 'lastGame1AtHomeAway', 'lastGame2WinAway', 'lastGame2AtHomeAway', 'lastGame3WinAway', 'lastGame3AtHomeAway', 'lastGame4WinAway', 'lastGame4AtHomeAway', 'lastGame5WinAway', 'lastGame5AtHomeAway'])

    # Iterate through each game where the teams have played at least 5 games.
    for index, game in games.iterrows():

        # Get the last five games for the team.
        last5games = games[(games['gamesPlayedAway'] < game.gamesPlayedAway) & ((games['teamAwayCode'] == game.teamAwayCode) | (games['teamHomeCode'] == game.teamAwayCode))].sort_values(by='gamesPlayedAway', ascending=False).head(5)

        if len(last5games) == 5:
            lastGame1WinAway = last5games["scoreAway"].iloc[0] < last5games["scoreHome"].iloc[0]
            lastGame2WinAway = last5games["scoreAway"].iloc[1] < last5games["scoreHome"].iloc[1]
            lastGame3WinAway = last5games["scoreAway"].iloc[2] < last5games["scoreHome"].iloc[2]
            lastGame4WinAway = last5games["scoreAway"].iloc[3] < last5games["scoreHome"].iloc[3]
            lastGame5WinAway = last5games["scoreAway"].iloc[4] < last5games["scoreHome"].iloc[4]
            #print(lastGame1WinAway, lastGame2WinAway, lastGame3WinAway, lastGame4WinAway, lastGame5WinAway)

            lastGame1AtHomeAway = last5games["teamAwayCode"].iloc[0] == game.teamAwayCode
            lastGame2AtHomeAway = last5games["teamAwayCode"].iloc[1] == game.teamAwayCode
            lastGame3AtHomeAway = last5games["teamAwayCode"].iloc[2] == game.teamAwayCode
            lastGame4AtHomeAway = last5games["teamAwayCode"].iloc[3] == game.teamAwayCode
            lastGame5AtHomeAway = last5games["teamAwayCode"].iloc[4] == game.teamAwayCode
            #print(lastGame1AtHomeAway, lastGame2AtHomeAway, lastGame3AtHomeAway, lastGame4AtHomeAway, lastGame5AtHomeAway)

            # Update the row with the history
            data.loc[index] = [lastGame1WinAway, lastGame1AtHomeAway, lastGame2WinAway, lastGame2AtHomeAway, lastGame3WinAway, lastGame3AtHomeAway, lastGame4WinAway, lastGame4AtHomeAway, lastGame5WinAway, lastGame5AtHomeAway]

    # Convert types         
    data['lastGame1WinAway'] = data['lastGame1WinAway'].astype('bool')
    data['lastGame2WinAway'] = data['lastGame2WinAway'].astype('bool')
    data['lastGame3WinAway'] = data['lastGame3WinAway'].astype('bool')
    data['lastGame4WinAway'] = data['lastGame4WinAway'].astype('bool')
    data['lastGame5WinAway'] = data['lastGame5WinAway'].astype('bool')
    data['lastGame1AtHomeAway'] = data['lastGame1AtHomeAway'].astype('bool')
    data['lastGame2AtHomeAway'] = data['lastGame2AtHomeAway'].astype('bool')
    data['lastGame3AtHomeAway'] = data['lastGame3AtHomeAway'].astype('bool')
    data['lastGame4AtHomeAway'] = data['lastGame4AtHomeAway'].astype('bool')
    data['lastGame5AtHomeAway'] = data['lastGame5AtHomeAway'].astype('bool')

    # Add results to the dataset
    games = pd.merge(games, data, left_index=True, right_index=True)
    
    games[['teamHomeCode', 'teamAwayCode', 'lastGame1WinHome', 'lastGame1AtHomeHome', 'lastGame2WinHome', 'lastGame2AtHomeHome', 'lastGame3WinHome', 'lastGame3AtHomeHome', 'lastGame4WinHome', 'lastGame4AtHomeHome', 'lastGame5WinHome', 'lastGame5AtHomeHome']].sample(5)

In [15]:
# test entry for stadium distance calc

coords_1 = (37.750267, -122.202853)
coords_2 = (34.04303865743706, -118.26711416244507)

geopy.distance.vincenty(coords_1, coords_2).miles

337.6783477035342

In [16]:
games["time"] = pd.to_datetime(games["time"])

In [17]:
if use_team_hot_encoding == True:
    homeTeam = pd.get_dummies(games["teamHomeCode"], prefix='team')
    awayTeam = pd.get_dummies(games["teamAwayCode"], prefix='team')

    homeTeam = homeTeam.replace({0:np.nan})
    awayTeam = awayTeam.replace({0:np.nan})

    teams = homeTeam.fillna(awayTeam).fillna(0).astype('bool')
    # teams[['team_NYK', 'team_CLE', 'team_ATL', 'team_BOS']].head(2)
    
    games = pd.concat([games, teams], axis=1)
    
    games[['teamHomeCode', 'teamAwayCode', 'team_NYK', 'team_CLE']].head(2)

In [18]:
if use_team_home_away_hot_encoding == True:
    homeTeam = pd.get_dummies(games["teamHomeCode"], prefix='hometeam').astype('bool')
    awayTeam = pd.get_dummies(games["teamAwayCode"], prefix='awayteam').astype('bool')

    games = pd.concat([games, homeTeam, awayTeam], axis=1)
    
    games[['teamHomeCode', 'teamAwayCode', 'hometeam_NYK', 'hometeam_CLE', 'awayteam_NYK', 'awayteam_CLE']].head(2)

In [19]:
if (use_team_hot_encoding == True) | (use_team_home_away_hot_encoding == True) | (use_team_keys == False):
    games = games.drop(["teamAwayId", "teamHomeId"], axis=1)

In [20]:
games = games.drop(["teamAwayCode", "teamHomeCode"], axis=1)

In [21]:
games = games.drop(["location", "scoreQuarters", "time"], axis=1)

In [22]:
games["homeWin"] = games["scoreHome"] > games["scoreAway"]

In [23]:
games = games.drop(["scoreAway", "scoreHome"], axis=1)

In [24]:


# Remove any games that have less than a 5 game history.
if calc_last_5_games == True:
    games = games[(pd.notnull(games['lastGame1WinHome'])) & (pd.notnull(games['lastGame1AtHomeAway']))]



In [25]:
games[games.isnull().T.any().T]

,id,oddsBet365Away,oddsBet365Home,season,gamesPlayedHome,gamesPlayedAway,totalGamesHome,totalWinsHome,homeGamesHome,homeWinsHome,...,lastGame1AtHomeAway,lastGame2WinAway,lastGame2AtHomeAway,lastGame3WinAway,lastGame3AtHomeAway,lastGame4WinAway,lastGame4AtHomeAway,lastGame5WinAway,lastGame5AtHomeAway,homeWin


In [26]:
#msk = np.random.rand(len(games)) < 0.8
#msk

train = games
#test = games[~msk]

 
temp=train

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for col in  train.columns:
    
    if train[col].dtype=='object' or  train[col].dtype=='bool':
        train[col]=le.fit_transform(train[col])

In [ ]:
train.head(3)

In [32]:

Y_train = train["homeWin"]
X_train = train.drop(["id","homeWin"], axis=1)
X=np.array(X_train)
y=np.array(Y_train)

In [ ]:
X_train.columns

In [38]:
 
import xgboost  as xgb

In [39]:
xgb.XGBClassifier()

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [ ]:
model=xgb.XGBClassifier(n_estimators=250)

In [ ]:
model.fit(np.array(X),y)


In [ ]:
import  matplotlib
from matplotlib  import pyplot as plt


In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
xgb.plot_importance(model, max_num_features=50, height=0.8, ax=ax)
plt.show()


In [ ]:
 ##########################################Checking accuracy  on  5 cross   validations###############################

In [41]:
from sklearn.metrics import accuracy_score

In [36]:
from sklearn.model_selection import train_test_split

In [ ]:
for cross  in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=cross)
    model=xgb.XGBClassifier(n_estimators=250)
    model.fit(X_train,y_train)
    pred=model.predict(X_test)
    print(accuracy_score(y_test,pred))
    
     
    

In [ ]:
np.unique(pred,return_counts=True)

In [ ]:
from sklearn.ensemble import  RandomForestClassifier
for cross  in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=cross)
    model=RandomForestClassifier(n_estimators=100)
    model.fit(X_train,y_train)
    pred=model.predict(X_test)
    print(accuracy_score(y_test,pred))
    
     
    

In [ ]:
forest=model
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()

In [44]:
import numpy as np
import GPy
import GPyOpt
from xgboost import XGBClassifier
from sklearn.cross_validation import cross_val_score
y=y.astype(int)

In [ ]:

def f(parameters):
    parameters = parameters[0]
    score = -cross_val_score(
                XGBClassifier(learning_rate=parameters[0],
                              max_depth=int(parameters[2]),
                              n_estimators=int(parameters[3]),
                              gamma=int(parameters[1]),
                              min_child_weight = parameters[4]), 
                X, y, scoring='accuracy').mean()
    score = np.array(score)
    return score
baseline = -cross_val_score(XGBClassifier(), X, y, scoring='accuracy').mean()
baseline

In [ ]:
bounds = [
            {'name': 'learning_rate', 'type': 'continuous', 'domain': (1e-5, 1)},
            {'name': 'gamma', 'type': 'continuous', 'domain': (1e-5, 5)},
            {'name': 'max_depth', 'type': 'discrete', 'domain': (1, 50)},
            {'name': 'n_estimators', 'type': 'discrete', 'domain': (1, 1000)},
            {'name': 'min_child_weight', 'type': 'discrete', 'domain': (1, 10)}
         ]

np.random.seed(777)
optimizer = GPyOpt.methods.BayesianOptimization(f=f, domain=bounds,
                                                acquisition_type ='MPI',
                                                acquisition_par = 0.1,
                                                exact_eval=True)
max_iter = 50
max_time = 60
optimizer.run_optimization(max_iter, max_time)

In [ ]:

optimizer.plot_convergence()

In [ ]:
np.set_printoptions(suppress=True)
optimizer.X[np.argmin(optimizer.Y)]

In [ ]:
np.min(optimizer.Y)

In [ ]:
for cross  in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=cross)
    model=xgb.XGBClassifier(n_estimators=1000,learning_rate=0.33589572,gamma=4.8989112,
                           max_depth= 50,min_child_weight=10)
    model.fit(X_train,y_train)
    pred=model.predict(X_test)
    print(accuracy_score(y_test,pred))
    
     
    

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
xgb.plot_importance(model, max_num_features=50, height=0.8, ax=ax)
plt.show()


In [ ]:
########################################Adding  new features ############################################################

In [ ]:
X

In [ ]:
###########################################correlation###################################################################

In [ ]:
np.corrcoef(X)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=2)

In [ ]:
new_train=pca.fit_transform(X)

In [ ]:
np.corrcoef(new_train)

In [ ]:
temp_train1=np.hstack((new_train,X))

In [ ]:
for cross  in range(5):
    X_train, X_test, y_train, y_test = train_test_split(temp_train1, y, test_size=0.33, random_state=cross)
    model=xgb.XGBClassifier(n_estimators=1000,learning_rate=0.33589572,gamma=4.8989112,
                           max_depth= 50,min_child_weight=10)
    model.fit(X_train,y_train)
    pred=model.predict(X_test)
    print(accuracy_score(y_test,pred))
    
     

In [ ]:
from sklearn.decomposition import TruncatedSVD

In [ ]:
svd = TruncatedSVD(n_components=10, n_iter=1000, random_state=42)

In [ ]:
new_train=svd.fit_transform(X)

In [ ]:
np.corrcoef(new_train)

In [ ]:
temp_train2=np.hstack((new_train,temp_train1))

In [ ]:
for cross  in range(5):
    X_train, X_test, y_train, y_test = train_test_split(temp_train2, y, test_size=0.33, random_state=cross)
    model=xgb.XGBClassifier(n_estimators=1000,learning_rate=0.33589572,gamma=4.8989112,
                           max_depth= 50,min_child_weight=10)
    model.fit(X_train,y_train)
    pred=model.predict(X_test)
    print(accuracy_score(y_test,pred))
    
     

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
new_train = TSNE(n_components=3).fit_transform(X)

In [ ]:
new_train.shape

In [ ]:
np.corrcoef(new_train[:,0],new_train[:,2])

In [ ]:
temp_train3=np.hstack((new_train,X))

In [ ]:
for cross  in range(5):
    X_train, X_test, y_train, y_test = train_test_split(temp_train3, y, test_size=0.33, random_state=cross)
    model=xgb.XGBClassifier(n_estimators=1000,learning_rate=0.33589572,gamma=4.8989112,
                           max_depth= 50,min_child_weight=10)
    model.fit(X_train,y_train)
    pred=model.predict(X_test)
    print(accuracy_score(y_test,pred))
    
     

In [ ]:
from sklearn.ensemble import  RandomForestClassifier
for cross  in range(5):
    X_train, X_test, y_train, y_test = train_test_split(temp_train3, y, test_size=0.33, random_state=cross)
    model=RandomForestClassifier(n_estimators=500)
    model.fit(X_train,y_train)
    pred=model.predict(X_test)
    print(accuracy_score(y_test,pred))
    
     
    

In [27]:
one_hot_encoded_training_predictors = pd.get_dummies(temp)

In [30]:
one_hot_encoded_training_predictors.shape

(1230, 626)

In [31]:
big_train=one_hot_encoded_training_predictors

In [33]:
big_train= big_train.drop(["id","homeWin"], axis=1)

In [34]:
big_train.shape

(1230, 624)

In [136]:
#for cross  in range(5):
#    X_train, X_test, y_train, y_test = train_test_split(big_train, y, test_size=0.2, random_state=cross)
#    model=xgb.XGBClassifier(n_estimators=5000,learning_rate=0.33589572,gamma=4.8989112,
#                           max_depth= 200,min_child_weight=10)
#    model.fit(X_train,y_train)
#    pred=model.predict(X_test)
#    print(accuracy_score(y_test,pred))
    
     

In [47]:

def f(parameters):
    parameters = parameters[0]
    score = -cross_val_score(
                XGBClassifier(learning_rate=parameters[0],
                              max_depth=int(parameters[2]),
                              n_estimators=int(parameters[3]),
                              gamma=int(parameters[1]),
                              min_child_weight = parameters[4]), 
                X, y, scoring='accuracy').mean()
    score = np.array(score)
    return score
baseline = -cross_val_score(XGBClassifier(), big_train, y, scoring='accuracy').mean()
baseline

-0.60898316937410624

In [54]:
bounds = [
            {'name': 'learning_rate', 'type': 'continuous', 'domain': (1e-5, 1)},
            {'name': 'gamma', 'type': 'continuous', 'domain': (1e-5, 5)},
            {'name': 'max_depth', 'type': 'discrete', 'domain': (1, 200)},
            {'name': 'n_estimators', 'type': 'discrete', 'domain': (1, 5000)},
            {'name': 'min_child_weight', 'type': 'discrete', 'domain': (1, 10)}
         ]

np.random.seed(777)
optimizer = GPyOpt.methods.BayesianOptimization(f=f, domain=bounds,
                                                acquisition_type ='MPI',
                                                acquisition_par = 0.1,
                                                exact_eval=True)
max_iter = 50
max_time = 60
optimizer.run_optimization(max_iter, max_time)

In [55]:
np.min(optimizer.Y)

-0.62928861075236497

In [56]:
np.set_printoptions(suppress=True)
optimizer.X[np.argmin(optimizer.Y)]

array([    0.35028689,     4.88600234,   200.        ,  5000.        ,
          10.        ])

In [129]:
import pandas
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.layers import Dense, Dropout, Convolution1D, Flatten, MaxPooling1D,BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

In [130]:
big_train.shape

(1230, 624)

In [133]:
 
def create_baseline():
    
    model = Sequential()
    
    model.add(Dense(128, input_dim=624, kernel_initializer='normal', activation='relu'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
 
    
    model.add(Dropout(0.1))
 
    model.add(Dense(64, kernel_initializer='normal', activation='tanh'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(Dropout(0.2))
 
    
    
    
      
    model.add(Dense(32, kernel_initializer='normal', activation='relu'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(Dropout(0.09))
    
    model.add(Dense(16, kernel_initializer='normal', activation='relu'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(Dropout(0.5))
    
    model.add(Dense(32, kernel_initializer='normal', activation='relu'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(Dropout(0.15))
    
    
    model.add(Dense(64, kernel_initializer='normal', activation='tanh'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    
    model.add(Dense(128, input_dim=624, kernel_initializer='normal', activation='relu'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [134]:
 
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, np.array(big_train), y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Results: 56.08% (6.40%)


In [135]:
####################################Averaging   first   attempt######################################################

In [139]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=cross)

In [138]:
X.shape

(1230, 43)

In [140]:
    model1=xgb.XGBClassifier(n_estimators=1000,learning_rate=0.33589572,gamma=4.8989112,
                           max_depth= 50,min_child_weight=10)
    model2=RandomForestClassifier(n_estimators=250)

In [141]:
    model1.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=4.8989112, learning_rate=0.33589572,
       max_delta_step=0, max_depth=50, min_child_weight=10, missing=None,
       n_estimators=1000, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [143]:
model2.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=250, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [144]:
pred1=model1.predict_proba(X_test)

In [146]:
pred2=model2.predict_proba(X_test)

In [148]:
avg=(pred1+pred2)/2

In [162]:
from sklearn.metrics  import log_loss
accuracy_score(y_test,np.round(avg[:,1]))

0.63008130081300817

In [65]:
y=y.reshape(y.shape[0],1)

In [73]:
damian=big_train

In [75]:
damian['y']=y

In [76]:
damian.to_csv('NBA_DATA.csv',index=False)

In [83]:
big_train=big_train.drop('y',axis=1)